In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [10]:
# Veriyi yükleyin
data = pd.read_csv("TCGA_LUAD_miRNA_expression_disease_status.csv", header=0, index_col=0)

# Özellikleri ve hedef değişkeni tanımlayın
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Veriyi eğitim ve test setlerine ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Özellikleri standardize edin
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Random Forest Regressor için Grid Search ile en iyi n_estimators değerini bulalım
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500]
}

model = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# En iyi parametreleri ve model performansını yazdırın
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")

# Test seti üzerinde tahmin yapın ve hatayı hesaplayın
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_estimators': 500}
Mean Squared Error: 0.019901600000000002


In [14]:
# Feature importance
feature_importances = best_model.feature_importances_


# Tüm özellik ve önem değerlerini dosyaya yazdırma
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
importance_df.to_csv('LUAD_miRNA_random_forest_selected_features.txt', sep='\t', index=False)


